In [28]:
import os
import pymssql

import pandas as pd
import numpy as np
import locale

from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate


load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')


'pt_BR.UTF-8'

In [19]:
descontos_sql = ''' 

SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pm.Descricao as descricao,
       gp.Nome as nome,
       processo.dbo.fn_Exe_RetornaValorAtualizado(ed.IdDebito) as valor_multa,
       (
       	SELECT top 1 vspfr.nome_orgao
       	FROM BdDIP.dbo.vwSiaiPessoalFolhaResumida vspfr 
       	WHERE vspfr.cpf  = gp.Documento COLLATE SQL_Latin1_General_CP1_CI_AS
              ORDER BY ano, mes DESC
       ) as orgao,
       ed.CodigoStatusDivida as status_divida
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) = 'DESCONTO EM FOLHA'
'''
processos_descontos = pd.read_sql(descontos_sql, conn)

/tmp/ipykernel_66896/3368704005.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_descontos = pd.read_sql(descontos_sql, conn)


In [20]:
processos_descontos['processo'] = processos_descontos['numero_processo'] + '/' + processos_descontos['ano_processo'].astype(str)
processos_descontos['valor'] = processos_descontos['valor_multa'].apply(lambda x: locale.currency(x, grouping=True))

In [21]:
processos_descontos

,numero_processo,ano_processo,assunto,descricao,nome,valor_multa,orgao,status_divida,processo,valor
0,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,José Miguel de Menezes Júnior,3135.7967,PREFEITURA MUNICIPAL DE SAO TOME,13.0,000088/2021,"R$ 3.135,80"
1,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,José Miguel de Menezes Júnior,4824.3031,PREFEITURA MUNICIPAL DE SAO TOME,13.0,000088/2021,"R$ 4.824,30"
2,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,José Miguel de Menezes Júnior,4826.6852,PREFEITURA MUNICIPAL DE SAO TOME,2.0,000088/2021,"R$ 4.826,69"
3,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOSINALDO AMARO DE LIMA,3439.5992,CÂMARA MUNICIPAL DE SÃO TOMÉ,1.0,000088/2021,"R$ 3.439,60"
4,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOSINALDO AMARO DE LIMA,3752.2783,CÂMARA MUNICIPAL DE SÃO TOMÉ,3.0,000088/2021,"R$ 3.752,28"
5,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOSINALDO AMARO DE LIMA,4824.3031,CÂMARA MUNICIPAL DE SÃO TOMÉ,3.0,000088/2021,"R$ 4.824,30"
6,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOSINALDO AMARO DE LIMA,5547.9485,CÂMARA MUNICIPAL DE SÃO TOMÉ,13.0,000088/2021,"R$ 5.547,95"
7,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOSINALDO AMARO DE LIMA,5789.1638,CÂMARA MUNICIPAL DE SÃO TOMÉ,3.0,000088/2021,"R$ 5.789,16"
8,002033,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,HAROLDO GOMES DA SILVA,5612.4217,PREFEITURA MUNICIPAL DE PARNAMIRIM,1.0,002033/2024,"R$ 5.612,42"
9,002033,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,HAROLDO GOMES DA SILVA,6479.3088,PREFEITURA MUNICIPAL DE PARNAMIRIM,13.0,002033/2024,"R$ 6.479,31"


In [22]:
descontos_aberto = processos_descontos[processos_descontos.status_divida == 1]

In [25]:
descontos_aberto

,numero_processo,ano_processo,assunto,descricao,nome,valor_multa,orgao,status_divida,processo,valor
3,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOSINALDO AMARO DE LIMA,3439.5992,CÂMARA MUNICIPAL DE SÃO TOMÉ,1.0,000088/2021,"R$ 3.439,60"
8,002033,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,HAROLDO GOMES DA SILVA,5612.4217,PREFEITURA MUNICIPAL DE PARNAMIRIM,1.0,002033/2024,"R$ 5.612,42"
13,002035,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,HAROLDO GOMES DA SILVA,5831.3906,PREFEITURA MUNICIPAL DE PARNAMIRIM,1.0,002035/2024,"R$ 5.831,39"
18,002138,2019,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,ABEL BELARMINO DE AMORIM FILHO,21336.6252,PREFEITURA MUNICIPAL DE RAFAEL GODEIRO,1.0,002138/2019,"R$ 21.336,63"
19,002138,2019,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,LUDMILA CARLOS AMORIM DE ARAÚJO ROSADO,21336.6252,PREFEITURA MUNICIPAL DE RAFAEL GODEIRO,1.0,002138/2019,"R$ 21.336,63"
20,003368,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Alexandre Pereira de Araujo Montenegro(vereador),10233.0620,CÂMARA MUNICIPAL DE PENDÊNCIAS,1.0,003368/2024,"R$ 10.233,06"
26,003392,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,MANOEL MARCONE BEZERRA,8725.4755,PREFEITURA MUNICIPAL DE JANDAÍRA,1.0,003392/2024,"R$ 8.725,48"
27,004923,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,"SONYARA DE SOUZA RIBEIRO, PREFEITA",8267.5228,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1.0,004923/2024,"R$ 8.267,52"
28,011032,2016,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,CARLOS JOSÉ FERNANDES,222.9542,CÂMARA MUNICIPAL DE NATAL,1.0,011032/2016,"R$ 222,95"
29,011032,2016,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,CARLOS JOSÉ FERNANDES,3419.9964,CÂMARA MUNICIPAL DE NATAL,1.0,011032/2016,"R$ 3.420,00"


In [29]:
for desconto in descontos_aberto.itertuples():
    doc = DocxTemplate("templates/desconto_folha.docx")
    context = {
        'processo': desconto.processo,
        'assunto': desconto.assunto,
        'nome': desconto.nome,
        'orgao': desconto.orgao,
        'valor': desconto.valor
    }
    processo_underline = desconto.processo.replace('/', '_')
    doc.render(context)
    doc.save(f"saidas/desconto_folha/{processo_underline}.docx")
    print(f"Gerado arquivo para o processo {desconto.processo}")

Gerado arquivo para o processo 000088/2021
Gerado arquivo para o processo 002033/2024
Gerado arquivo para o processo 002035/2024
Gerado arquivo para o processo 002138/2019
Gerado arquivo para o processo 002138/2019
Gerado arquivo para o processo 003368/2024
Gerado arquivo para o processo 003392/2024
Gerado arquivo para o processo 004923/2024
Gerado arquivo para o processo 011032/2016
Gerado arquivo para o processo 011032/2016
